In [1]:
from idaes.dmf import DMF, resource, tables

# Read table
tbl = tables.Table()
tbl.read_csv("pitzer_1.csv")
tbl.data

,T,P,G0/RT H2O,G0/RT NaCl,A phi,B0,B1,10^3xC
0,0,1.0,-23.4638,-13.836,0.3767,0.0493,0.2462,2.58
1,10,1.0,-22.9100,-13.871,0.3821,0.0619,0.2612,1.68
2,20,1.0,-22.4049,-13.885,0.3882,0.0714,0.2723,0.99
3,25,1.0,-22.1689,-13.886,0.3915,0.0754,0.2770,0.70
4,30,1.0,-21.9433,-13.885,0.3949,0.0788,0.2814,0.44
...,...,...,...,...,...,...,...,...
187,260,1000.0,-16.9080,-12.516,0.6512,0.0781,0.4769,-1.57
188,270,1000.0,-16.8466,-12.432,0.6691,0.0732,0.4873,-1.36
189,280,1000.0,-16.7905,-12.339,0.6874,0.0677,0.4978,-1.11
190,290,1000.0,-16.7393,-12.237,0.7063,0.0614,0.5084,-0.83


In [2]:
tbl.units_dict

{'T ': 'C',
 'P ': 'bar',
 'G0/RT H2O ': '',
 'G0/RT NaCl ': '',
 'A phi ': '(kg/mol^0.5',
 'B0 ': 'kg/mol',
 'B1 ': 'kg/mol',
 '10^3xC ': '(kg/mol)^2'}

In [3]:
r = resource.Resource(name="pitzer-1", type_=resource.ResourceTypes.tabular)

In [4]:
source = {"date": "1984", "doi": "10.1063/1.555709", "source": 'Pitzer, Kenneth S., J. Christopher Peiper, and R. H. Busey. "Thermodynamic properties of aqueous sodium chloride solutions." Journal of Physical and Chemical Reference Data 13.1 (1984): 1-102.'}

In [5]:
source

{'date': '1984',
 'doi': '10.1063/1.555709',
 'source': 'Pitzer, Kenneth S., J. Christopher Peiper, and R. H. Busey. "Thermodynamic properties of aqueous sodium chloride solutions." Journal of Physical and Chemical Reference Data 13.1 (1984): 1-102.'}

In [6]:
r.v["sources"] = [source]

In [7]:
tbl.add_to_resource(r)

In [8]:
pwd

'C:\\Users\\DanGunter\\src\\idaes\\dangunter\\idaes-pse\\idaes\\dmf\\tests\\data_files\\pitzer'

In [9]:
dmf = DMF("dmf_workspace", create=True)

2021-08-07 13:21:29,750 [INFO] idaes.dmf.workspace: Using existing path for new DMF workspace: C:\Users\DanGunter\src\idaes\dangunter\idaes-pse\idaes\dmf\tests\data_files\pitzer\dmf_workspace
2021-08-07 13:21:29,752 [INFO] idaes.dmf.workspace: Create new configuration at 'C:\Users\DanGunter\src\idaes\dangunter\idaes-pse\idaes\dmf\tests\data_files\pitzer\dmf_workspace\config.yaml'
2021-08-07 13:21:29,758 [INFO] idaes.dmf.dmfbase: Saving configuration location to: C:\Users\DanGunter/.dmf


In [10]:
dmf.add(r)

'dd426f175535457eb7a469102d827b40'

In [11]:
dmf.update()

1

In [13]:
dmf2 = DMF("dmf_workspace")

2021-08-07 13:21:42,610 [INFO] idaes.dmf.dmfbase: Saving configuration location to: C:\Users\DanGunter/.dmf


In [14]:
fetched_resource = dmf2.find_one(name="pitzer-1")

In [15]:
fetched_resource.sources

[{'date': '1984',
  'doi': '10.1063/1.555709',
  'source': 'Pitzer, Kenneth S., J. Christopher Peiper, and R. H. Busey. "Thermodynamic properties of aqueous sodium chloride solutions." Journal of Physical and Chemical Reference Data 13.1 (1984): 1-102.'}]

In [16]:
fetched_resource.data

{'table': {'T ': {'units': 'C',
   'values': [0,
    10,
    20,
    25,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    110,
    120,
    130,
    140,
    150,
    160,
    170,
    180,
    190,
    200,
    210,
    220,
    230,
    240,
    250,
    260,
    270,
    280,
    290,
    300,
    0,
    10,
    20,
    25,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    110,
    120,
    130,
    140,
    150,
    160,
    170,
    180,
    190,
    200,
    210,
    220,
    230,
    240,
    250,
    260,
    270,
    280,
    290,
    300,
    0,
    10,
    20,
    25,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    110,
    120,
    130,
    140,
    150,
    160,
    170,
    180,
    190,
    200,
    210,
    220,
    230,
    240,
    250,
    260,
    270,
    280,
    290,
    300,
    0,
    10,
    20,
    25,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    110,
    120,
    130,
    140

In [17]:
tbl2 = tables.Table.get_from_resource(fetched_resource)

In [19]:
tbl2.data

,T,P,G0/RT H2O,G0/RT NaCl,A phi,B0,B1,10^3xC
0,0,1.0,-23.4638,-13.836,0.3767,0.0493,0.2462,2.58
1,10,1.0,-22.9100,-13.871,0.3821,0.0619,0.2612,1.68
2,20,1.0,-22.4049,-13.885,0.3882,0.0714,0.2723,0.99
3,25,1.0,-22.1689,-13.886,0.3915,0.0754,0.2770,0.70
4,30,1.0,-21.9433,-13.885,0.3949,0.0788,0.2814,0.44
...,...,...,...,...,...,...,...,...
187,260,1000.0,-16.9080,-12.516,0.6512,0.0781,0.4769,-1.57
188,270,1000.0,-16.8466,-12.432,0.6691,0.0732,0.4873,-1.36
189,280,1000.0,-16.7905,-12.339,0.6874,0.0677,0.4978,-1.11
190,290,1000.0,-16.7393,-12.237,0.7063,0.0614,0.5084,-0.83


In [20]:
tbl2.units_dict

{'T ': 'C',
 'P ': 'bar',
 'G0/RT H2O ': '',
 'G0/RT NaCl ': '',
 'A phi ': '(kg/mol^0.5',
 'B0 ': 'kg/mol',
 'B1 ': 'kg/mol',
 '10^3xC ': '(kg/mol)^2'}